In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib as mpl
import anndata as anndata
import pathlib
import sys

In [ ]:
sample_name = 'OMIX002713'
use_fcgr1a = True

In [ ]:
def calculate_and_plot_score(adata: anndata.AnnData, gene_list, score_name, **kwargs):
    sc.tl.score_genes(ad, gene_list = gene_list, score_name=score_name)
    sc.pl.spatial(ad, color = score_name, img_key=None, size = 1.6, save = '_{}.png'.format(score_name), return_fig = False, **kwargs)


In [ ]:
database_path = pathlib.Path('../../../shared/databases/')

In [ ]:
if use_fcgr1a:
    MAC_GENES = ['CD163', 'LYZ', 'FCGR1A'] # APOE'CD68' is expressed almost everywhere: 'MRC1' not enough
else:
    MAC_GENES = ['CD163', 'LYZ']

numof_genes = 150
KOvsWT_down_genelist = pd.read_csv(database_path / '{}_KO--over--WT_Down-Regulated.csv'.format(numof_genes), header=0).gene_name.str.upper()
KOvsWT_up_genelist = pd.read_csv(database_path / '{}_KO--over--WT_Up-Regulated.csv'.format(numof_genes), header=0).gene_name.str.upper()
KOvsWT_down_genelist = KOvsWT_down_genelist.str.upper()
KOvsWT_up_genelist = KOvsWT_up_genelist.str.upper()


In [ ]:
print(len(KOvsWT_down_genelist))

In [ ]:
sample_folder = pathlib.Path('../data/raw')

path_to_results = pathlib.Path('../') / 'results'
path_to_results.mkdir(parents=True, exist_ok=True)
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=100, facecolor='white', figsize = (4,4), dpi_save=300, frameon = False, vector_friendly = False)
sc.settings.figdir = path_to_results
spatial_dot_size = 1.6
spot_size = 1.6
spatial_transparency = 1

In [ ]:
adata = sc.read(sample_folder / "OMIX002713.h5ad")

In [ ]:
len(adata.obs['orig.ident'].cat.categories)

In [ ]:
adata.obs['orig.ident'].cat.categories

In [ ]:
sc.pl.umap(adata, color = ['celltype', 'labels', 'histology'])

In [ ]:
sc.pl.umap(adata, color = MAC_GENES)

In [ ]:
sc.pl.umap(adata, color = MAC_GENES)

In [ ]:
sc.tl.score_genes(adata, gene_list=KOvsWT_down_genelist, score_name='KOvsWT_down_score')

In [ ]:
sc.tl.score_genes(adata, gene_list=KOvsWT_up_genelist, score_name='KOvsWT_up_score')

In [ ]:
sc.pl.umap(adata, color = 'KOvsWT_down_score', cmap='Oranges')

In [ ]:
sc.pl.umap(adata, color = 'KOvsWT_up_score', cmap='Oranges')

In [ ]:
adata.obs['KOvsWT_up_minus_down_score'] = adata.obs['KOvsWT_up_score'] - adata.obs['KOvsWT_down_score']

In [ ]:
vmax = adata.obs['KOvsWT_up_minus_down_score'].abs().quantile(0.99)
vmin = -vmax
sc.pl.umap(adata, color = 'KOvsWT_up_minus_down_score', vmax = vmax, vmin = vmin, cmap='bwr')

# Fig 1F

In [ ]:
df = sc.get.obs_df(adata, keys = ['SPP1', 'CXCL10', 'CD74'])

In [ ]:
#define conditions
conditions = [
    (df['CXCL10'] > 0) & (df['SPP1'] > 0),
    (df['CXCL10'] > 0) & (df['SPP1'] == 0),
    (df['CXCL10'] == 0)& (df['SPP1'] > 0),
    (df['CXCL10'] == 0)& (df['SPP1'] == 0),
]

#define results
results = ['CXCL10 & SPP1', 'CXCL10', 'SPP1', 'none']

#create new column based on conditions in column1 and column2
adata.obs['CXCL10_SPP1'] = np.select(conditions, results)

In [ ]:
Cxcl10_Spp1_colors= {'CXCL10':'#40E0D0', 'SPP1':'#FF1493', 'CXCL10 & SPP1': '#5D29A6', 'none': '#F2F2F2'}

In [ ]:
sc.pl.violin(adata, keys='KOvsWT_up_minus_down_score', groupby='CXCL10_SPP1',  palette = Cxcl10_Spp1_colors, stripplot = False, inner = 'box', rotation = 45)

In [ ]:
#define conditions
conditions = [
    (df['CD74'] > 0) & (df['SPP1'] > 0),
    (df['CD74'] > 0) & (df['SPP1'] == 0),
    (df['CD74'] == 0)& (df['SPP1'] > 0),
    (df['CD74'] == 0)& (df['SPP1'] == 0),
]

#define results
results = ['CD74 & SPP1', 'CD74', 'SPP1', 'none']

#create new column based on conditions in column1 and column2
adata.obs['CD74_SPP1'] = np.select(conditions, results)

In [ ]:
CD74_SPP1_colors= {'CD74':'#40E0D0', 'SPP1':'#FF1493', 'CD74 & SPP1': '#5D29A6', 'none': '#F2F2F2'}

In [ ]:
sc.pl.violin(adata, keys='KOvsWT_up_minus_down_score', groupby='CD74_SPP1', palette = CD74_SPP1_colors, stripplot = False, inner = 'box',rotation = 90)